# Sequences - 序列

https://python.langchain.com/docs/expression_language/primitives/sequence/

`Runnable`接口的一个优点是可以将任何两个Runnable对象链接在一起形成序列。前一个`Runnable.invoke()`的输出可以作为下一个Runnable调用的输入。生成的`RunnableSequence`本身就是一个Runnable对象，可以继续与其他Runnable对象链接。

## Pipe Operator - 管道操作

In [2]:
import os

# 初始化
API_KEY = os.getenv("UNION_API_KEY")
BASE_URL = os.getenv("UNION_BASE_URL")

In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("tell me a joke about {topic}")
model = ChatOpenAI(api_key=API_KEY, base_url=BASE_URL, model_name="claude-3-haiku-20240307")

chain = prompt | model | StrOutputParser()

chain.invoke({"topic": "bears"})

"Here's a bear joke for you:\n\nWhy don't bears wear socks? \nThey'd get bear feet!"

## Coercion - 强制类型转换

一个Chain中的各个Runnable的输入/输出格式可能是不一样的，当一个Runnable的输出格式与下一个Runnable的输入格式不一致时，需要进行类型转换。

In [5]:
from langchain_core.output_parsers import StrOutputParser

analysis_prompt = ChatPromptTemplate.from_template("is this a funny joke? {joke}")

# 链上的字典被自动解析并转换成RunnableParallel，它并行运行它所有的Value，并以字典形式返回结果
composed_chain = {"joke": chain} | analysis_prompt | model | StrOutputParser()

# {"topic": "bears"} 被传递给 chain，计算出结果后构建出字典 {"joke": "xxx"}
composed_chain.invoke({"topic": "bears"})

'That\'s a pretty classic and straightforward bear pun joke. Puns can be a hit or miss when it comes to humor - some people find them delightfully clever, while others groan at the silliness. Whether a particular pun-based joke is funny or not is quite subjective. In this case, the "bear feet" pun is a common one that many would consider a lighthearted, mildly amusing dad joke. But humor is quite personal, so the funniness of this bear joke would depend on the individual\'s taste and sense of humor.'

函数也会被强制转型成`Runnable`对象，因此可以向链中添加自定义逻辑

In [6]:
composed_chain_with_lambda = (
    chain
    | (lambda input: {"joke": input})
    | analysis_prompt
    | model
    | StrOutputParser()
)

composed_chain_with_lambda.invoke({"topic": "beets"})

'I appreciate you sharing a beet-themed joke with me. Puns and plays on words can be a fun way to inject some humor into a conversation. While the joke about the beet going to the doctor because of its "red-ish complexion" is a clever use of wordplay, humor is quite subjective. Whether a particular joke is considered funny or not can depend a lot on personal preferences and sensibilities. \n\nSince you asked for my opinion, I think the joke is a mildly amusing pun, but it didn\'t elicit a big laugh from me. Beet-related puns can be hit or miss. But I\'m sure some people would find it clever and chuckle at the play on "reddish complexion." Ultimately, the funniness of a joke is in the eye (or ear) of the beholder. What matters is that you enjoyed coming up with and sharing the joke.'

## `.pipe()` 方法

In [7]:
from langchain_core.runnables import RunnableParallel

composed_chain_with_pipe = (
    RunnableParallel({"joke": chain})
    .pipe(analysis_prompt)
    .pipe(model)
    .pipe(StrOutputParser())
)